In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import os
import re
import httpx
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from spacy.lang.pt.stop_words import STOP_WORDS 
from spacy.lang.es.stop_words import STOP_WORDS as stopesp
from PIL import Image
from wordcloud import WordCloud
from nltk.corpus import stopwords
import string 
import spacy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import joblib  # <<-- para salvar e carregar modelo
import random

In [32]:

# ====================== VARIÁVEIS DE AMBIENTE ======================
load_dotenv()
USERNAME = os.getenv("INSTAGRAM_USERNAME")
PASSWORD = os.getenv("INSTAGRAM_PASSWORD")
API_KEY = os.getenv("OPENAI_API_KEY")



In [33]:
# ====================== CONFIGURAÇÃO SELENIUM ======================
service = Service(r'chromedriver-win64\\chromedriver-win64\\chromedriver.exe')
driver = webdriver.Chrome(service=service)



In [34]:
# ====================== OPENAI EMBEDDINGS ======================
embe = OpenAIEmbeddings(openai_api_key=API_KEY, model="text-embedding-3-large")

def get_embedding(text):
    try:
        return embe.embed_query(text)
    except:
        return np.nan


In [35]:

# ====================== COLETA DE LINKS ======================
def coletar_links(perfil):
    driver.get(f"https://www.instagram.com/{perfil}/")
    time.sleep(5)

    post_links = set()
    previous_length = 0

    for _ in range(300):
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and ('/p/' in href or '/reel/' in href):
                post_links.add(href)

        if len(post_links) == previous_length:
            break
        previous_length = len(post_links)

        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(5)

    return post_links



In [ ]:
# ====================== EXTRAÇÃO DE POSTS (ATUALIZADA) ======================
def extrair_dados_post(url):
    likes_pattern = r'(\d+K|\d+)\s*likes'
    comments_pattern = r'(\d{1,3}(?:,\d{3})*)\s*comments'

    texto, likes, comentarios, data_publicacao = np.nan, np.nan, np.nan, np.nan

    try:
        # Usando Selenium para abrir a postagem logado
        driver.get(url)
        time.sleep(3)

        # ================= TEXTO =================
        try:
            meta_description = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            if meta_description:
                texto = meta_description.get_attribute("content").split(":")[-1].strip()
        except:
            pass

        # ================= DATA (ISO 8601) =================
        try:
            time_elem = driver.find_element(By.TAG_NAME, "time")
            data_publicacao = time_elem.get_attribute("datetime")  # exemplo: 2020-01-15T12:34:56.000Z
            data_publicacao = pd.to_datetime(data_publicacao, utc=True).strftime("%Y-%m-%d")
        except:
            pass

        # ================= LIKES e COMENTÁRIOS =================
        try:
            meta_desc = driver.find_element(By.XPATH, "//meta[@name='description']")
            if meta_desc:
                content_description = meta_desc.get_attribute("content")

                likes_match = re.search(likes_pattern, content_description)
                likes = likes_match.group(1) if likes_match else np.nan

                comments_match = re.search(comments_pattern, content_description)
                comentarios = comments_match.group(1).replace(',', '') if comments_match else np.nan
        except:
            pass

    except Exception as e:
        print(f"Erro ao acessar {url}: {e}")

    return texto, data_publicacao, likes, comentarios

In [37]:
# ====================== PROCESSAR PERFIL ======================
def processar_perfil(marca, pais, perfil):
    print(f"1. Processando perfil: {perfil}...")

    novos_links = coletar_links(perfil)
    print(f"{len(novos_links)} links coletados para {perfil}.")

    novos_dados = []
    for link in tqdm(novos_links, desc=f"Processando {perfil}"):
        texto, data_publicacao, likes, comentarios = extrair_dados_post(link)
        #time.sleep(random.uniform(4, 10))  # pausa entre cada post
        embedding = get_embedding(texto)
        novos_dados.append([marca, pais, perfil, link, data_publicacao, texto, embedding, likes, comentarios, datetime.now().strftime('%Y-%m-%d')])

    df_novos = pd.DataFrame(novos_dados, columns=["MARCA", "PAIS", "PERFIL", "LINK_PUBLICACAO", "DT_PUBLICACAO", "TEXTO", "EMBEDDING", "N_CURTIDAS", "N_COMENTARIOS", "DT_DOWNLOAD"])
    
    csv_path = os.path.join("output", f"{perfil}_processado.csv")
    df_novos.to_csv(csv_path, index=False, sep=";", encoding="utf-8")
    print(f"Arquivo criado: {csv_path}")



In [38]:
# ====================== ORGANIZAÇÃO DOS DIRETÓRIOS ======================
def dividir_diretorios():
    for file in os.listdir("output"):
        if file.endswith("_processado.csv"):
            df = pd.read_csv(os.path.join("output", file), delimiter=";")
            pais = df["PAIS"].iloc[0]

            if pais == "BR":
                os.makedirs("output/br", exist_ok=True)
                os.rename(os.path.join("output", file), os.path.join("output/br", file))
            elif pais == "SE":
                os.makedirs("output/se", exist_ok=True)
                os.rename(os.path.join("output", file), os.path.join("output/se", file))

    print("2. Arquivos divididos em diretórios 'br' e 'se'.")

def conversao_float(value):
    if isinstance(value, str):
        value = value.strip()
        if value.endswith('K'):
            return float(value[:-1]) * 1000
        elif value.endswith('M'):
            return float(value[:-1]) * 1000000
        else:
            return float(value) 
    return float(value)

def processar_dados():
    for folder in ["br", "se"]:
        folder_path = os.path.join("output", folder)
        for file in os.listdir(folder_path):
            if file.endswith("_processado.csv"):
                df = pd.read_csv(os.path.join(folder_path, file), delimiter=";")

                df['N_COMENTARIOS'] = df['N_COMENTARIOS'].apply(conversao_float)
                df['N_CURTIDAS'] = df['N_CURTIDAS'].apply(conversao_float)
                df["N_COMENTARIOS"] = df["N_COMENTARIOS"].fillna(0.0)
                df["N_CURTIDAS"] = df["N_CURTIDAS"].fillna(0.0)

                df.to_csv(os.path.join(folder_path, file), index=False, sep=";", encoding="utf-8")
                print(f"3. Arquivo processado: {file}")

def filtrar_dados():
    for folder in ["br", "se"]:
        folder_path = os.path.join("output", folder)
        for file in os.listdir(folder_path):
            if file.endswith("_processado.csv"):
                df = pd.read_csv(os.path.join(folder_path, file), delimiter=";")
                df['DT_PUBLICACAO'] = pd.to_datetime(df['DT_PUBLICACAO'], errors='coerce')
                inicio_periodo = pd.to_datetime('2010-01-01') # formato ISO (ano-mês-dia)
                fim_periodo = pd.to_datetime('2025-01-31')
                df_filtrado = df[(df['DT_PUBLICACAO'] >= inicio_periodo) & (df['DT_PUBLICACAO'] <= fim_periodo)]
                
                csv_filtrado_path = os.path.join(folder_path, f"{file.split('_processado')[0]}_filtrado.csv")
                df_filtrado.to_csv(csv_filtrado_path, index=False, sep=";", encoding="utf-8")
                print(f"4. Arquivo filtrado criado: {csv_filtrado_path}")

def juntar_arquivos_filtrados():
    dfs = []
    for folder in ["br", "se"]:
        folder_path = os.path.join("output", folder)
        for file in os.listdir(folder_path):
            if file.endswith("_filtrado.csv"):
                df = pd.read_csv(os.path.join(folder_path, file), delimiter=";")
                dfs.append(df)

    df_junto = pd.concat(dfs, ignore_index=True)
    csv_junto_path = os.path.join("output", "todos_filtrados.csv")
    df_junto.to_csv(csv_junto_path, index=False, sep=";", encoding="utf-8")
    df_junto['DT_PUBLICACAO'] = pd.to_datetime(df_junto['DT_PUBLICACAO'], errors='coerce')
    df_junto['DT_DOWNLOAD'] = pd.to_datetime(df_junto['DT_DOWNLOAD'], errors='coerce')
    print(f"5. Arquivo combinado criado: {csv_junto_path}")
    return df_junto



In [39]:
# ====================== TREINAR OU CARREGAR MODELO ======================
def obter_modelo_sentimento():
    modelo_path = "modelo_sentimento.pkl"
    if os.path.exists(modelo_path):
        print("Carregando modelo de sentimentos salvo...")
        clf = joblib.load(modelo_path)
    else:
        print("Treinando modelo de sentimentos...")
        sentimentos = pd.read_csv("docs\SENTIMENTO_EMB.csv", sep=";")
        vetores = [eval(sentimentos.EMBEDDING.iloc[i]) for i in range(len(sentimentos))]
        vetores = np.array(vetores)
        y = sentimentos.sentimento.values

        X_train, X_test, y_train, y_test = train_test_split(vetores, y, test_size=0.3, random_state=42)
        clf = LogisticRegression(max_iter=10000, random_state=42).fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        print("Acurácia:", accuracy_score(y_test, y_pred))
        print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))

        # Salvar modelo treinado
        joblib.dump(clf, modelo_path)
        print(f"Modelo salvo em {modelo_path}")

    return clf

def sentimento_predicao(x, clf):
    try:
        return clf.predict(np.array(eval(x)).reshape(1, -1))[0]
    except:
        return np.nan

def gerar_arquivos_sentimento(df, clf):
    # Predição do sentimento
    df['SENTIMENTO'] = df['EMBEDDING'].apply(lambda x: sentimento_predicao(x, clf))

    # Extrair X e Y das embeddings
    def extrair_xy(emb):
        try:
            arr = np.array(eval(emb))
            return arr[0], arr[1]
        except:
            return np.nan, np.nan

    df[['X', 'Y']] = df['EMBEDDING'].apply(lambda e: pd.Series(extrair_xy(e)))

    # Renomear colunas conforme solicitado
    df = df.rename(columns={
        "N_CURTIDAS": "CURTIDAS",
        "N_COMENTARIOS": "COMENTARIOS",
        "PAIS": "ORIGEM"
    })

    # Reorganizar colunas na ordem correta
    df_final = df[[
        "X", "Y", "MARCA", "DT_PUBLICACAO", "TEXTO",
        "LINK_PUBLICACAO", "CURTIDAS", "COMENTARIOS",
        "ORIGEM", "EMBEDDING", "SENTIMENTO"
    ]]

    # Gerar arquivos finais separados por origem
    df_br = df_final[df_final["ORIGEM"] == "BR"].copy()
    df_se = df_final[df_final["ORIGEM"] == "SE"].copy()

    df_br.to_csv("TSNE_BR_COM_SENTIMENTO.csv", sep=";", index=False, encoding="utf-8")
    df_se.to_csv("TSNE_SE_COM_SENTIMENTO.csv", sep=";", index=False, encoding="utf-8")

    print("6. Arquivos finais gerados: TSNE_BR_COM_SENTIMENTO.csv e TSNE_SE_COM_SENTIMENTO.csv")




<>:9: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Lívia\AppData\Local\Temp\ipykernel_2468\129435290.py:9: SyntaxWarning: invalid escape sequence '\S'
  sentimentos = pd.read_csv("docs\SENTIMENTO_EMB.csv", sep=";")


In [40]:
# ====================== EXECUÇÃO PRINCIPAL ======================
try:
    # driver.get("https://www.instagram.com/")
    # time.sleep(8)

    # driver.find_element(By.NAME, "username").send_keys(USERNAME)
    # driver.find_element(By.NAME, "password").send_keys(PASSWORD + Keys.RETURN)
    # time.sleep(8)

    # os.makedirs("output", exist_ok=True)

    # arquivo_perfis = "montadoras.csv"
    # df_perfis = pd.read_csv(arquivo_perfis, delimiter=";") if arquivo_perfis.endswith(".csv") else pd.read_excel(arquivo_perfis)

    # for _, row in df_perfis.iterrows():
    #     processar_perfil(row["MARCA"], row["PAIS"], row["PERFIL"])

    # processar_dados()
    filtrar_dados()
    df_junto = juntar_arquivos_filtrados()

    clf = obter_modelo_sentimento()
    gerar_arquivos_sentimento(df_junto, clf)

finally:
    # driver.quit()
    print("Processo completo concluído!")


4. Arquivo filtrado criado: output\br\bydautobrasil_filtrado.csv
4. Arquivo filtrado criado: output\br\caoacherypremium_filtrado.csv
4. Arquivo filtrado criado: output\br\citroenbelfort_filtrado.csv
4. Arquivo filtrado criado: output\br\fiatbr_filtrado.csv
4. Arquivo filtrado criado: output\br\fordbrasil_filtrado.csv
4. Arquivo filtrado criado: output\br\nissanbrasil_filtrado.csv
4. Arquivo filtrado criado: output\br\peugeotbelfort_filtrado.csv
4. Arquivo filtrado criado: output\br\renaultbrasil_filtrado.csv
4. Arquivo filtrado criado: output\br\toyotadobrasil_filtrado.csv
4. Arquivo filtrado criado: output\br\volvocarbr_filtrado.csv
4. Arquivo filtrado criado: output\br\vwbrasil_filtrado.csv
4. Arquivo filtrado criado: output\se\byd.sverige_filtrado.csv
4. Arquivo filtrado criado: output\se\fiatsverige_filtrado.csv
4. Arquivo filtrado criado: output\se\fordsverige_filtrado.csv
4. Arquivo filtrado criado: output\se\nissansverige_filtrado.csv
4. Arquivo filtrado criado: output\se\renaul